### Cahabón Fog Capture Model 

##### Analysis Set up 
This calculation of Fog Interception is based on the model developed by Mark Mulligan and Sophia Burke for FIESTA (Fog Interception for the Enhancement of Streamflow in Tropical Areas). 
Mulligan, M. and Burke, S. (2005). Fiesta fog interception for the enhancement of streamflow in tropical areas
final technical report for kcl/amibiotek contirbution to dfid frp project r799. 
(https://www.researchgate.net/publication/237574142_Final_Technical_Report_DFID-FRP_Project_no_R7991_Hydrological_impacts_of_converting_tropical_montane_cloud_forest_to_pasture_with_initial_reference_to_northern_Costa_Rica) 

###### Data
INSIVUMEH - Meteorological data from Estación Cobán 1991-2022. Retreived from public information request to INSIVUMEH via email


#### Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import datetime
import math
import pandas as pd
#colors https://www.webucator.com/article/python-color-constants-module/

#### Input Files and Constants 

In [2]:
#Files
#Inputs
spatial_file = 'FIESTA_spatial_yalijux_cahabon_frst.csv'#'QGIS_LandClass_fracs.csv'
timeseries_file = 'INSIVUMEH_Coban_1970_synth.csv'
#timeseries_file = 'Meteo_yearmonthavgs.csv' #'INSIVUMEH_Meteo.csv' #INSIVUMEH_Coban = INSIVUMEH_Coban_in[3256:-1, :]

#Outputs
Fog_Evap_Budg_filename = 'Fog_Evap_Budg_70.npy'
fog_val_filename = 'fog_val_70.npy'
LCL_val_filename = 'LCL_val_70.npy'

#Constants
MSLP = 1 #[mb] Mean sea level pressure
kWh_MJ = 3.6 #Conversion from kWh to MJ multiply
SecondsInMonth = np.array([2678400, 2419200, 2678400, 2592000, 2678400, 2592000, 2678400, 2678400, 2592000, 2678400, 2592000, 2678400, 2505600])
droplet_size = 7.5 #dorplet size  [um]
DropTermVel = 9 #m/s. Average terminal velocity of raindrop. TBD --> adjust depending on cloud height and type of rain. 
#Define LAI of each land use: 
LAI_TMCF = 4.5
LAI_Clear = 2
LAI_Pine = 2.5 
#Leaf Self Shading for each land use
lss_TMCF = -0.7 * 0.3 * 10
lss_Pine = -0.7 * 0.3 * 10
lss_Clear  = -0.7 * 6.0 *  0.5

#Scenarios
scenario = 'no_dLULC'
#scenario = 'totaldeforestation'
#scenario = 'totalreforestation'
#scenario = 'dLULC_1991_2016' 




#### Run model

In [12]:
Fog_Evap_Budg_Scn2 = run_FIESTA_model(droplet_size, scenario, spatial_file, timeseries_file, Fog_Evap_Budg_filename, fog_val_filename, LCL_val_filename)
    

#### Functions

In [11]:
def load_timeseries_file(timeseries_file):
    INSIVUMEH_Coban = np.loadtxt(timeseries_file, delimiter=',')

    #INSIVUMEH_Coban = np.loadtxt('Meteo_only_temp_var.csv', delimiter=',')
    #Año	Mes	Día	LR	T_max	T_min	T_mean	Rh	CloudFrac13	Rad24	Rad6	Rad12	Rad18	
    # Wind_d	Wind_v	Press	ENSO	Precip	EvapPot
    ts_len = len(INSIVUMEH_Coban)
    Year  = INSIVUMEH_Coban[:, 0]
    Month = INSIVUMEH_Coban[:, 1]
    Day   = INSIVUMEH_Coban[:, 2]
    LR    = INSIVUMEH_Coban[:, 3]
    T_max = INSIVUMEH_Coban[:, 4]
    T_min = INSIVUMEH_Coban[:, 5]
    T_avg = INSIVUMEH_Coban[:, 6]
    RH    = INSIVUMEH_Coban[:, 7]
    Cld   = INSIVUMEH_Coban[:, 8] / 13 
    Rad24 = INSIVUMEH_Coban[:, 9]
    Rad6  = INSIVUMEH_Coban[:, 10]
    Rad12 = INSIVUMEH_Coban[:, 11]
    Rad18 = INSIVUMEH_Coban[:, 12]
    Wnd_d = INSIVUMEH_Coban[:, 13]
    Wnd_spd =INSIVUMEH_Coban[:, 14]
    Presh = INSIVUMEH_Coban[:, 15]
    #Presh = (Presh_in - (113)) * 1.333223874 #Shifting and converting to mb
    #ENSO  = INSIVUMEH_Coban[:, 16]
    Prcp  = INSIVUMEH_Coban[:, 17]
    #Epot  = INSIVUMEH_Coban[:, 18]
    return ts_len, Year, Month, Day, LR, T_max, T_min, T_avg, RH, Cld, Rad24, Rad6, Rad12, Rad18, Wnd_d, Wnd_spd, Presh, Prcp

def load_spatial_file(spatial_file):
    Gridcells = np.loadtxt(spatial_file, delimiter=',')
    LUclass_len = len(Gridcells)
    #Elev	Aspect	Area	Area3D	Slope	Frac_clr	Frac_frst	Frac_refrst	Frac_defrst
    Elev        = Gridcells[:,0]
    Aspect      = Gridcells[:,1]
    Slope       = Gridcells[:,2]
    CellSize    = Gridcells[:,3]
    Area3d      = Gridcells[:,4]
    Frac_clr    = Gridcells[:,5]
    Frac_frst   = Gridcells[:,6]
    Frac_refrst = Gridcells[:,7]
    Frac_defrst = Gridcells[:,8]
    return LUclass_len, Elev, Aspect, Slope, CellSize, Area3d, Frac_clr, Frac_frst, Frac_refrst, Frac_defrst


def LULC_scenario(scenario, LUclass_len, ts_len, Frac_clr, Frac_defrst, Frac_frst, Frac_refrst):
    # 2016 Conditions, No Land Use Change
    if scenario == 'no_dLULC': 
        TreeFrac    = np.zeros([LUclass_len, ts_len])
        FracClear   = np.zeros([LUclass_len, ts_len])
        FracTMCF    = np.zeros([LUclass_len, ts_len])
        FracPine    = np.zeros([LUclass_len, ts_len])
        for k in range(LUclass_len):
            FracClear[k, :]   = Frac_clr[k] + Frac_defrst[k]
            FracTMCF[k, :]    = Frac_frst[k]
            FracPine[k, :]    = Frac_refrst[k]
        TreeFrac = FracTMCF + FracPine 

    #Complete Deforestation, No Land Use Change
    if scenario == 'totaldeforestation':
        TreeFrac    = np.ones([LUclass_len, ts_len]) * 0.05
        FracClear   = np.ones([LUclass_len, ts_len]) * 0.95
        FracTMCF    = np.ones([LUclass_len, ts_len]) * 0.025
        FracPine    = np.ones([LUclass_len, ts_len]) * 0.025

    #Complete Reforestation, No Land Use Change
    if scenario == 'totalreforestation':
        TreeFrac    = np.ones([LUclass_len, ts_len]) * 0.975
        FracClear   = np.ones([LUclass_len, ts_len]) * 0.025
        FracTMCF    = np.ones([LUclass_len, ts_len]) * 0.95
        FracPine    = np.ones([LUclass_len, ts_len]) * 0.025

    #Gradual Change in Deforestation and Reforeattion from 1991 to 2016 (real conditions)
    if scenario == 'dLULC_1991_2016': 
        # t0 
        #    --> FracClear = Frac_clr + Frac_reforested
        #    --> FracPine  = 0
        #    --> TreeTMCF  = Fracfrst 
        #    --> TreeFrac  = FracTMCF + FracPine
        # t_end
        #    --> FracClear = Frac_clr + Frac_deforested
        #    --> FracPine  = Frac_reforested
        #    --> TreeTMCF  = Fracfrst - Frac_deforested
        #    --> TreeFrac  = FracTMCF + FracPine

        FracClear   = np.zeros([LUclass_len, ts_len])
        FracTMCF    = np.zeros([LUclass_len, ts_len])
        FracPine    = np.zeros([LUclass_len, ts_len])

        for k in range(LUclass_len):
            #At t = 0 
            frac0_Clear = Frac_clr[k]
            frac0_TMCF  = Frac_frst[k]
            frac0_Pine  = 0
            #At t = end
            fracN_Clear = Frac_clr[k] + Frac_defrst[k]
            fracN_TMCF  = Frac_frst[k] - Frac_defrst[k]
            fracN_Pine  = Frac_refrst[k]
            for i in range(ts_len):
                FracClear[k,i] = frac0_Clear + (fracN_Clear - frac0_Clear) / ts_len * i
                FracTMCF[k,i]  = frac0_TMCF  + (fracN_TMCF  - frac0_TMCF)  / ts_len * i
                FracPine[k,i]  = frac0_Pine  + (fracN_Pine  - frac0_Pine)  / ts_len * i
        TreeFrac = FracTMCF + FracPine
    return FracClear, FracTMCF, FracPine, TreeFrac

def Forest_Edge(Treefraction, Cellarea):
    #FIESTA = The formula relates two different grid cell size due to the data sources used in FIESTA, 
    #         this is not necessary in our model. 
    #         forestedgefrac = -3 * 10 ** (-5) * Treefraction ** 2 + 0.0036 * Treefraction
    #         forestedgelenm = forestedgefrac * ((Cellarea ** 2)/(25 * 25)) * 100
    #         emergentedgelenm = (0.05 * Treefraction)*((Cellarea ** 2)/(25 * 25)) * 100 #calculated as 5% of fraction of area covered by trees
    #         forestedgelenfacingm = forestedgelenm / 4
    #         emergentedgelenfacing = emergentedgelenm / 4
    #MDP = 
    forestedgefrac = -0.0001 * ((Treefraction*100) ** 2) + 0.0144 * (Treefraction*100) #From Figure 59, (Mulligan & Burke, 2005)
    forestedgelenm = forestedgefrac * np.sqrt(Cellarea) * 4 #edge faction in a fraction of the total edge of the cell.
    emergentedgelenm = (0.15 * Treefraction) * np.sqrt(Cellarea) * 4
    forestedgelenfacingm = forestedgelenm / 4
    emergentedgelenfacingm = emergentedgelenm / 4
    # Output units = meters
    return forestedgelenfacingm, emergentedgelenfacingm  

def Sedimentation_Surface_Area(Frac_frst, Frac_clr, Frac_pine, LAI_frst, LAI_clr, LAI_pine, lss_frst, lss_pine, lss_clr):
    #Sedimentation Surface area 
    # F = Key assumption : That the whole unshaded (one sided) leaf surface area is available for sedimentation (deposition)
    #Surface area for each land use:
    #FIESTA uses these constants: 
    #lss_frst = -0.7 * 0.3 * 10, leaf self shading forest 
    #lss_pine = -0.7 * 0.3 * 10, leaf slef shading pine 
    #lss_clr  = -0.7 * 6.0 *  0.5, leaf self shading clear
    TrappingSfcArea_frst = (1 - (np.exp(lss_frst))) 
    TrappingSfcArea_pine = (1 - (np.exp(lss_pine)))
    TrappingSfcArea_clr = (1 - (np.exp(lss_clr)))
    #Total deposition area per grid cell
    DepositionFrac = (Frac_frst * TrappingSfcArea_frst * LAI_frst) + (Frac_pine * TrappingSfcArea_pine * LAI_pine) + (Frac_clr * TrappingSfcArea_clr * LAI_clr)
    # Output Units = Fraction
    return DepositionFrac 

def ET_LU(Frac_frst, Frac_clr, Frac_pine, LAI_frst, LAI_clr, LAI_pine):
    #Intercepted Energy Fractions
    # Key assumption : That evapotranspiration is effectively modelled at 
    # this coarse spatial and temporal scale from consideration of energy 
    # availablility and atmospheric demand for water only. Leaf area is 
    # sufficient to represent plant processes and aerodynamic resistances can safely be ignored. 
    ExpLAI_frst = 1-np.exp(-0.7*max(1,LAI_frst))
    ExpLAI_pine = 1-np.exp(-0.7*max(1,LAI_pine))
    ExpLAI_clr  = 1-np.exp(-0.7*max(1,LAI_clr))
    
    EtFrac_frst = Frac_frst * ExpLAI_frst
    EtFrac_pine = Frac_pine * ExpLAI_pine
    EtFrac_clr  = Frac_clr  * ExpLAI_clr
    
    EtFrac_total = EtFrac_frst + EtFrac_pine + EtFrac_clr
    
    # Output Units = Fraction
    return EtFrac_total 

def FogSettlingVel(droplet_size):
    #Fog Settling
    #Eq 2. FIESTA =  That fog settling occurs under calm conditions and upwards fog 
    #          turbulent diffusion is limited compared with this downward flux, 
    #          Fog settling velocity is calculated according to Stokes Law based 
    #          on the mean particle size for fog. 
    # MDP   =  Same as FIESTA, Stokes law 
    FogSettlingVel = (980 * ((droplet_size/10000)**2)*(1-0.0013))/(18*0.000185)
    # Output units = m/s
    return FogSettlingVel 

def T_elev_shift(Elev_cell, T_var_C=-0.6, Elev_station=1320):
    # Temperature
    # INSIVUMEH
    # FIESTA = regular diurnal range variation. 
    # MDP    = same as FIESTA.
    #T_var_C = -0.6 C/100m
    T_shift = (Elev_cell - Elev_station) * T_var_C / 100
    return T_shift

def T_lucc_shift(Frac_frst, T_mean, T_var_C=-1,):
    # Temperature
    T_lu_shift = T_mean + Frac_frst * T_var_C
    return T_lu_shift

def Temps(Hour, T_max, T_min, T_mean_in, Elev, Frac_frst):
    # Inputs: 
    #       - Hour = 1, 2, 3, or 4: the four timesteps through the day. 
    #       - T_max, T_min, T_mean = Read from INSIVUMEH
    #       - T_shift = change in temperature for cell's elevation band
    # Output: 
    #       - Tmp = Temperature at time step, adjusted for elevation
    T_lu_shift = T_lucc_shift(Frac_frst, T_mean_in)
    T_mean = T_lu_shift + T_elev_shift(Elev, T_var_C=-0.6, Elev_station=1320)
    DiurnalTRange = T_max-T_min
    if(Hour == 1): 
        Tmp = T_mean - 0.25 * DiurnalTRange
        AirRising = 0
    elif (Hour == 2):
        Tmp = T_mean
        AirRising = 1
    elif (Hour == 3):
        Tmp = T_mean + 0.25 * DiurnalTRange
        AirRising = 1
    else:
        Tmp = T_mean
        AirRising = 0
    #Output Units = Degrees Celsius
    return Tmp, AirRising

def Saturated_vapur_press(Tmp, RH):
    # Dewpoint
    # Inputs: 
    #       - Tmp = tamperature (C)
    #       - RH = relative humidity (%)
    # Outputs: 
    #       - Es = saturated vapour pressure (mb)
    #       - E  = vapour pressure (mb) 
    Es = np.exp(26.66082-0.0091379024*(Tmp + 273.15)-(6106.396/(Tmp+273.15))) #Saturated Vapour Pressure (mb)
    E = (RH/100)*Es #Vapour Pressure (mb)
    # Output units = mb
    return E


def Air_density(MSLP, Tmp, E):
    # Air Density 
    # Inputs: 
    #      - MSLP = mean sea level pressure (mb)
    #      - Tmp  = Temperature
    #      - E    = Vapour Pressure
    # Outputs: 
    #      - 
    AirDensity = (MSLP*100)/((Tmp+273.15)*287) #Air density [kg/m3]
    AH = (E*100)/((Tmp+273.15)*461.5) #Absolute humidity [kg/m3]
    # Output units = Air density [kg/m3], 
    return AirDensity, AH 

def LiqWaterContent(AH_n, AH_max, maxLWC=0.0002):
    # Liquid Water Content
    # Cloud liquid water content is proportional to absolute atmospheric humidity. 
    # maxLWC = 0.0002 #usually observed maximum AH [kg/m3]
    # LWC = (AH_n/AH_max)* maxLWC
    LWC = 0.0002
    # Output Units = kg/m3
    return LWC 

def Dewpoint(E):
    # Function to calculate Dewpoint Temperature (Td) [C]
    # Inputs: 
    #       - E = Vapour Pressure [mb]
    # Outpouts:
    #       - Td = Dew Point Temperature [C]
    btemp = (26.66082 - np.log(E))
    Td    = ((btemp-np.sqrt((btemp**2)-223.1986))/0.0182758048)-273.15
    # Output Units = Degrees Celsius
    return Td

def LiftingCondensationLevel(Tmp, Td, MSLP, Elevation):
    # Function to calculate Lifting Condensation Level (LCL) [m]
    # Inputs: 
    #      - Tmp = Ground Temperature [C]
    #      - Td  = Dew point temperature [C]
    # Outputs: 
    #      - LCL = Lifting Condensation level [masl]
    #      - fog = Presence of fog (0 or 1) [-]
    # LCL_mb = (1/(((Tmp-Td)/223.15)+1)**3.5)*MSLP
    # LCL_masl   = max((44.3308-4.94654*((LCL_mb*100)**0.190263))*1000, 0)
    LCL_coban = 125 * (Tmp - Td)
    LCL_masl = LCL_coban + 1320
    if (LCL_masl < Elevation): 
        fog = 1
    else: 
        fog = 0
    return LCL_masl, fog 

def EffectiveSolarRad(CloudFreqFrac, fog, SolarR_toa):
    if(fog==1):
        TransmissionLoss = (CloudFreqFrac*0.678)+((1-CloudFreqFrac)*-0.143)
    else:
        TransmissionLoss = (CloudFreqFrac*0.525)+((1-CloudFreqFrac)*-0.143)
    SolarR_ground = SolarR_toa * (1 - TransmissionLoss)
    # Output units = W/m2
    return SolarR_ground 

def NetRad(SolarR_ground, Frac_frst, Frac_pine, Frac_clr, Month, LR, SecondsInMonth):
    # Key assumption from FIESTA: The solar to net radiation conversion functions measured under  
    #   forest and grassland are representative for larger areas and other covers of similar density.
    SIM = SecondsInMonth[int(Month)]
    if (Month == 2) and (LR == 1): #February, Leap Year 
        SIM = SecondsInMonth[12] 
    # Some empirical constants from FIESTA -->
    # Based on the linear relationship between solar and net radiation for forest and field sites in the study location in CR. 
    FIESTA_C_tree_b = -27.9
    FIESTA_C_tree_m = 0.9
    FIESTA_C_clr_b  = -27.5
    FIESTA_C_clr_m  = 0.8
    SolarWm = SolarR_ground #(SolarR_ground*1000000)/(SIM/2) #This is the function from FIESTA, but their input is in MJ, our input is kWh though
    # SolarWm = SolarR_ground / 3600
    NetMap_frst = ((Frac_frst)*(FIESTA_C_tree_b+(FIESTA_C_tree_m*SolarWm)))
    NetMap_pine = ((Frac_pine)*(FIESTA_C_tree_b+(FIESTA_C_tree_m*SolarWm)))
    NetMap_clr  = ((Frac_clr) *(FIESTA_C_clr_b +(FIESTA_C_clr_m *SolarWm)))
    NetMap  = NetMap_frst +  NetMap_pine + NetMap_clr
    return NetMap 

def WindSpeed(Wind_d, Wind_spd, Aspect):
    #Not following FIESTA. 
    Wind_dir_adj = Aspect - Wind_d
    Wind_spd_adj = Wind_spd * abs(np.cos(np.radians(Wind_dir_adj/2)))
    
    # aspect_dir = [[1, 315, 45], [2, 45, 135], [3, 135, 225], [4, 225, 315]] #slope aspect and bounding angles for perpendicular wind
    # Key Assumption: perpendicular aspect = 100%, parallel aspect = 50%, opposite aspect = 1
    # factor_perp = 1
    # factor_par  = 0.5
    # factor_opp  = 0.1
    # if (Wind_d > aspect_dir[int(Aspect)-1][1] and Wind_d < aspect_dir[int(Aspect)-1][1]): 
    #     Wind_spd_adj = Wind_spd * factor_perp
    #     Wind_dir_adj   = 0
    # elif Wind_d > (aspect_dir[int(Aspect)-1][1]-90) and Wind_d < aspect_dir[int(Aspect)-1][2] or Wind_d > (aspect_dir[int(Aspect)-1][2]) and Wind_d < (aspect_dir[int(Aspect)-1][2]+90):
    #     Wind_spd_adj = Wind_spd * factor_par
    #     Wind_dir_adj   = 45
    # else:
    #     Wind_spd_adj = Wind_spd * factor_opp
    #     Wind_dir_adj   = 90
    # Wind Speed - Next steps: 
    #   - Set wind exposure at higher elevations, due to prevailing wind direction perpendicular to catchment valley. 
    #   - Interpolate Cobán and Panzos weather station wind directions. 
    #   - Apply daily variation based on average daily range from ICOBN4 weather station
    # Output units = m/s
    return Wind_spd_adj, Wind_dir_adj

def RainMod(Prec, Wind_spd_adj, Wind_dir_adj, slope, DropTermVel):
    # Same as Fiesta Eq 15
    TanRainInclination = 0
    if Prec > 0:
        TanRainInclination = Wind_spd_adj/DropTermVel
        check = 1 + slope * TanRainInclination * math.cos(Wind_dir_adj)
        WindSlopeCorrectionFactor = max(check,0)
    else:
        TanRainInclination = 0
        WindSlopeCorrectionFactor = 0
    Prec_corrected = Prec * WindSlopeCorrectionFactor
    #Output 
    return Prec_corrected 

def EdgeImpactFlux(Wind_spd_adj, forestedgelenfacingm, emergentedgelenfacingm, LWC):
    #Copied from FIESTA
    #Eq 16. This equation calculates the kg/hr/cell of water passing through forest edges.
    reduc_factor = 0.6053 # is a factor which reduces wind speeds as a result of frictional losses with the forest
                        #  and is calculate from comparison of forest and pasture site vertical wind profiles.
    H_mean_forest_edge = 10 # represents the mean height of forest edges (10m)
    H_mean_emergent = 2 #is the average height of emergents above the surrounding canopy (1.5m)
    WindFlux_Edg = (Wind_spd_adj*reduc_factor*3600)*forestedgelenfacingm * H_mean_forest_edge
    EdgeImpactionFlux = LWC * WindFlux_Edg
    WindFlux_Emrg = (Wind_spd_adj*3600)*emergentedgelenfacingm * H_mean_emergent
    EmergentImpactionFLux = LWC * WindFlux_Emrg
    #Output units = kg/hr/gridcell
    return  EdgeImpactionFlux, EmergentImpactionFLux 

def FogDeposition(Wind_spd_adj, TreeFrac, CellSize, LWC, FogSettlingVel, CellTrueArea, EdgeImpactionFlux, EmergentImpactionFlux, AirRising, DepositionFrac, CloudFreqFrac, fog):
    #Copied from FIESTA
    #Eq 17. A fog inclination angle for fog inputs over forest and pasture is calculated, based on 
    #       their respective wind speeds. A vertical flux is calculated as the fog settling velocity 
    #       over the whole cell surface area (rather than any vertical catching surfaces). proportion 
    #       of fog inputs that are deposited rather than impacted depends upon the cosine of the fog 
    #       inclination angle over grassland and forest fractions.
    
    #Angles
    reduc_factor_forest = 0.6053 # is a factor which reduces wind speeds as a result of frictional losses with the forest
    reduc_factor_grass = 0.5030 # is a factor which reduces wind speeds as a result of frictional losses with the pasture
    ForestFogInclinationAngle = math.atan((Wind_spd_adj * reduc_factor_forest)/FogSettlingVel)
    PastureFogInclinationAngle = math.atan((Wind_spd_adj * reduc_factor_grass)/FogSettlingVel)
    #Proportions of Deposition vs Impaction 
    DeposProportion = ((math.cos(ForestFogInclinationAngle))*TreeFrac)+((math.cos(PastureFogInclinationAngle))*(1-TreeFrac))
    ImpactionProportion = 1 - DeposProportion

    
    #Total Deposition
    #Settling
    GravityFlux = (FogSettlingVel*3600)*CellTrueArea
    SettlingFlux = LWC * GravityFlux #[kg/gridcell]
    DeposInterc = fog * ( SettlingFlux * DeposProportion) * DepositionFrac
    
    #Eq 18. Vegetation Areas for Fog Interception
    #Grass
    Grass_height = 0.5
    WindFLux_Dep_Grass = (Wind_spd_adj*reduc_factor_grass*3600)*(1-TreeFrac)*CellSize*Grass_height
    GrassImpactionFlux = LWC * WindFLux_Dep_Grass #Units = [kg/cell]
    
    # AirRising 
    # In FIESTA, the parameter AirRising is true for situation where upwind elevations are greater than the downwind cell
    # In this adaptation, because mountain breeze dominates over the prevailing wind,AirRising is true during the day and false at night 
    ForestTrappingSfcArea = (1-np.exp((-0.7*0.3*(TreeFrac))/math.cos(ForestFogInclinationAngle)))
    PastureTrappingSfcArea = (1-np.exp((-0.7*6*(1-(TreeFrac)))/math.cos(PastureFogInclinationAngle)))
    #Impaction Flux
    ImpactionFrac = (AirRising * ForestTrappingSfcArea)
    ImpactionFlux = (EmergentImpactionFlux + EdgeImpactionFlux + GrassImpactionFlux)
    ImpactionInterc = fog * (ImpactionFlux * ImpactionProportion) * ImpactionFrac
    
    
    #Total Fog FLux
    FogInterc = DeposInterc + ImpactionInterc #kg/m2/hr
    FogIntmm = (FogInterc/CellTrueArea)*(CloudFreqFrac)

    return FogIntmm, DeposInterc

def Evap(NewTemp, NetMap, EtFrac_total):
    #Copied from FIESTA
    #Eq 21 = Thus evaporation is calculated on the basis of available energy and atmospheric 
    #        demand to give potential evaporation and this is then combined with the non self 
    #        shaded surface area available for the interception of radiation/evaporation of water to 
    #        give something closer to actual evaporation, which is responsive to vegetation type 
    #        and cover as well as climate conditions
    #Input Variables: 
    #        NewTemp = Air temp
    #        NetMap = Radiation [W/m2]
    #Output Variables: 
    #       ActEvap = Actual Evaporation 
    lat_hvap = 2.45 #Latent Heat of Vaporization [MJ/kg] 
    Ea = (611 * np.exp((17.27 * NewTemp)/(273.15+NewTemp)))/1000 #Vapor Pressure in [KPa]
    SlopeSatCurveK = (4098*Ea)/((273.15+NewTemp)**2) #Slope of the Saturation Vapour Pressure Curve [KPaC]
    PotEvap_m_d = (SlopeSatCurveK/(SlopeSatCurveK+0.066))*NetMap
    PotEvap = PotEvap_m_d * (60 * 60 / 1000000)
    if PotEvap > 0: 
        PotEvap_adj = PotEvap / lat_hvap
        ActEvap = PotEvap_adj * EtFrac_total
    else:
        PotEvap_adj = 0
        ActEvap = 0
    return ActEvap 

def Budgets(Prec, FogIntmm, ActEvap):
    #Copied from FIESTA
    #Eq. 22 
    Budget = ((Prec + FogIntmm)-ActEvap)
    return Budget 


# #Monthly, Hourly, Slope Variations for Solar Radiation
# rad_vars   = np.loadtxt('solar_rad_slope_scaling.csv', delimiter=',')
# # The solar radiation data for 4 time steps in the time series file is 
# # for south facing slopes, for N, E amd W, the values must be mutliplied by scaling factors in this file
# # Data columns--> (E, W, N, S) * (24, 6, 11, 17)
# #          E	E	E	E	W	W	W	W	N	N	N	N	S	S	S	S
# #         24	6	11	17	24	6	11	17	24	6	11	17	24	6	11	17
# # Data rows --> Jan to Dec
# #         J	F	M	A	M	J	J	A	S	O	N	D
# #Fill arrays with values, radS * scalar. 
# def rad_EWN(rad_S, Month):
#     #Building Solar Radiation Arrays for each slope direction:
#     rad_E = np.zeros(np.shape(rad_S))
#     rad_W = np.zeros(np.shape(rad_S))
#     rad_N = np.zeros(np.shape(rad_S))
#     rad = np.zeros(np.shape(rad_S))
#     for i in range(len(rad_S)):
#         for j in range(4):
#             rad_E[i, j] = rad_S[i, j] * rad_vars[int(Month[i])-1, j] * kWh_MJ
#         for j in range(4,8): 
#             rad_W[i, j] = rad_S[i, j] * rad_vars[int(Month[i])-1, j] * kWh_MJ
#         for j in range(8,12):
#             rad_N[i, j] = rad_S[i, j] * rad_vars[int(Month[i])-1, j] * kWh_MJ
#         for j in range(12,16):
#             rad_S[i, j] = rad_S[i, j] * rad_vars[int(Month[i])-1, j] * kWh_MJ
#     return rad_E, rad_W, rad_N, rad_S                                                                            

def run_FIESTA_model(droplet_size, scenario, spatial_file, timeseries_file, Fog_Evap_Budg_filename, fog_val_filename, LCL_val_filename):
    #load time series
    ts_len, Year, Month, Day, LR, T_max, T_min, T_avg, RH, Cld, Rad24, Rad6, Rad12, Rad18, Wnd_d, Wnd_spd, Presh, Prcp = load_timeseries_file(timeseries_file)
    
    #load gridcells
    LUclass_len, Elev, Aspect, Slope, CellSize, Area3d, Frac_clr, Frac_frst, Frac_refrst, Frac_defrst = load_spatial_file(spatial_file)
    
    #No temporal variation
    FogSettlingVelocity = FogSettlingVel(droplet_size)
    FracClear, FracTMCF, FracPine, TreeFrac = LULC_scenario(scenario, LUclass_len, ts_len, Frac_clr, Frac_defrst, Frac_frst, Frac_refrst)
    
    #initialize space-time 
    Fog_Evap_Budg = np.zeros((LUclass_len, ts_len, 3))
    fog_val = np.zeros((LUclass_len, ts_len, 4))
    LCL_val = np.zeros((LUclass_len, ts_len, 4))
    
    for k in range(len(Elev)):
        for i in range(len(Year)):
            Daily_deposition = np.zeros(4)
            Daily_evap = np.zeros(4)
            Daily_budget = np.zeros(4)
            for j in range(1, 5):
                forestedgelenfacingm, emergentedgelenfacingm = Forest_Edge(TreeFrac[k,i], CellSize[k])
                DepositionFrac   = Sedimentation_Surface_Area(FracTMCF[k,i], FracClear[k,i], FracPine[k,i], LAI_TMCF, LAI_Clear, LAI_Pine, lss_TMCF, lss_Pine, lss_Clear)
                EtFrac_total     = ET_LU(FracTMCF[k,i], FracClear[k,i], FracPine[k,i], LAI_TMCF, LAI_Clear, LAI_Pine)
                Tmp, AirRising   = Temps(j, T_max[i], T_min[i], T_avg[i], Elev[k], FracTMCF[k,i]) #Tmp [C]
                E                = Saturated_vapur_press(Tmp, RH[i]) #E [mb]
                AirDensity, AH   = Air_density(Presh[i], Tmp, E)
                LWC              = LiqWaterContent(AH, AH_max=0.02717, maxLWC=0.0002)
                Td               = Dewpoint(E) # Td [C]
                LCL, fog         = LiftingCondensationLevel(Tmp, Td, Presh[i], Elev[k])
                if   j == 1:
                    SolarR_ground = EffectiveSolarRad(Cld[i], fog, Rad24[i])
                elif j == 2:
                    SolarR_ground = EffectiveSolarRad(Cld[i], fog, Rad6[i])
                elif j == 3:
                    SolarR_ground = EffectiveSolarRad(Cld[i], fog, Rad12[i])
                elif j == 4:
                    SolarR_ground = EffectiveSolarRad(Cld[i], fog, Rad18[i]) # SolarR_ground [W/m2]
                NetMap            = NetRad(SolarR_ground, FracTMCF[k, i], FracPine[k, i], FracClear[k, i], Month[i], LR[i], SecondsInMonth) #NetMap [W/m2]
                
                Wind_spd_adj, Wind_dir_adj = WindSpeed(Wnd_d[i], Wnd_spd[i], Aspect[k])
                Prec_corrected    = RainMod(Prcp[i], Wind_spd_adj, Wind_dir_adj, Slope[k], DropTermVel)
                EdgeImpactionFlux, EmergentImpactionFLux = EdgeImpactFlux(Wind_spd_adj, forestedgelenfacingm, emergentedgelenfacingm, LWC)
                FogIntmm, DeposInterc = FogDeposition(Wind_spd_adj, TreeFrac[k, i], CellSize[k], LWC, FogSettlingVelocity, Area3d[k], EdgeImpactionFlux, EmergentImpactionFLux, AirRising, DepositionFrac, Cld[i], fog)
                ActEvap           = Evap(Tmp, NetMap, EtFrac_total)
                Budget            = Budgets(Prec_corrected, FogIntmm, ActEvap)
                #Store Outputs 
                Daily_deposition[j-1] = FogIntmm * 6 #6 hours per timestep
                Daily_evap[j-1] = ActEvap
                Daily_budget[j-1] = Budget
                fog_val[k, i, j-1] = fog
                LCL_val[k, i, j-1] = LCL
            Fog_Evap_Budg[k, i, 0] = np.sum(Daily_deposition)
            Fog_Evap_Budg[k, i, 1] = np.sum(Daily_evap)
            Fog_Evap_Budg[k, i, 2] = np.sum(Budget)

    Fog_Evap_Budg_Scn2 =  Fog_Evap_Budg
    TreeFrac_Scn2 = TreeFrac
    fog_valScn2 =  fog_val
    LCL_valScn2 = LCL_val 
    np.save(Fog_Evap_Budg_filename, Fog_Evap_Budg_Scn2)
    np.save(fog_val_filename, fog_valScn2)
    np.save(LCL_val_filename, LCL_valScn2)
    return Fog_Evap_Budg_Scn2
